<a href="https://colab.research.google.com/github/mmillet88/russian_therapy/blob/main/pre_processingSyntheticResponseDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading libraries

In [ ]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 1.2 MB 55.9 MB/s 
     |████████████████████████████████| 365 kB 72.4 MB/s 
     |████████████████████████████████| 6.6 MB 35.9 MB/s 
     |████████████████████████████████| 101 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 53.0 MB/s 
     |████████████████████████████████| 141 kB 55.6 MB/s 
     |████████████████████████████████| 115 kB 58.4 MB/s 
     |████████████████████████████████| 212 kB 59.7 MB/s 
     |████████████████████████████████| 127 kB 72.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
import pandas as pd 
import matplotlib.pyplot as plt

from transformers import AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)
import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
import gc

import datasets
from datasets import load_dataset
from datasets import DatasetDict

import numpy as np
from numpy.lib.function_base import average

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=cb65cf7be668d436c7d3ff44a81403399ca9310db1ba6ae5df0454cf43be95ea
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# Downloading a dataset

In [ ]:
import os
!mkdir /content/emp_chatbot
!mkdir emp_chatbot/data

mkdir: cannot create directory ‘/content/emp_chatbot’: File exists
mkdir: cannot create directory ‘emp_chatbot/data’: No such file or directory


In [ ]:
!ls

all_emotions_responses.json


In [ ]:
os.chdir('/content/emp_chatbot/data')

In [ ]:
# https://drive.google.com/file/d/121M9d19m7ljXaw0f5pggpHd9gsN2Hu3U/view?usp=sharing
# !gdown --id 1GaaG8UYKIHWSXR1X3OuxEfBpD2i-3qar
!gdown --id 1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp
# https://drive.google.com/file/d/1GaaG8UYKIHWSXR1X3OuxEfBpD2i-3qar/view?usp=sharing
# !gdown --id 1UhqyvObGgMl3_jEIkDiYJQEkQ7rFOXOc

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp
To: /content/emp_chatbot/data/all_emotions_responses.json
100% 2.38M/2.38M [00:00<00:00, 245MB/s]


In [ ]:
# data_files_EPru = {"train": "sadness_questions_follow_up.json"}
data_files_EPru = {"train": "all_emotions_responses.json"}
epru = load_dataset('json', data_files=data_files_EPru)

Using custom data configuration default-3a1f74a7752a7548


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3a1f74a7752a7548/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets.load import load_from_disk
ds = load_from_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset")

# Pre-computing Fluency and Embeddings




## Fluency/perplexity

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AutoModelWithLMHead, AutoTokenizer

device = "cuda"
model_id = "sberbank-ai/ruT5-base"
# model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
model_LM = AutoModelWithLMHead.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1002: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
columns = ds.column_names
print(columns)

['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'emotion', 'Would you like to attempt protocols for joy?']


In [ ]:
columns = ds.column_names
columns_indices = list(range(0,len(columns)))
perplexity_columns = [columns[index] for index in columns_indices if columns[index]!="emotion"]

In [ ]:
perplexity_columns

['Have you strongly felt or expressed any of the following emotions towards someone?',
 'Do you believe you should be the savior of someone else?',
 'Do you feel that you are trying to control someone?',
 'Do you see yourself as the victim, blaming someone else for how negative you feel?',
 'Are you always blaming and accusing yourself for when something goes wrong?',
 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?',
 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?',
 'Was this emotion triggered by a specific event?',
 'Was the event recent?',
 'Did you find protocol 6 distressing?',
 'Did you find protocol 11 distressing?',
 'Is it ok to ask additional questions?',
 'Would you like to attempt protocols for joy?']

In [ ]:
def calculate_perplexity(row, column_name):
  if row[column_name] != None and len(row[column_name]) != 0:
    encodings = tokenizer(row[column_name], return_tensors="pt")
    input_ids = encodings.input_ids.to(device)
    target_ids = input_ids.clone()
    with torch.no_grad():
      outputs = model_LM(input_ids, labels=target_ids)
      neg_log_likelihood = outputs[0]
    string_name = f"ppl_{column_name}"
    ppl = torch.exp(neg_log_likelihood)
    return {f'ppl_{column_name}': 1/ppl.item()}
  elif row[column_name] == None:
    return {f'ppl_{column_name}': 0}

In [ ]:
for column_name in perplexity_columns:
  ds = ds.map(lambda x: calculate_perplexity(x, column_name))

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

...

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

In [ ]:
# change dataset into pandas 
ds.set_format(type="pandas")
df = ds[:]
df.head()
# emotions = set(df['emotion'].tolist())

,Have you strongly felt or expressed any of the following emotions towards someone?,Do you believe you should be the savior of someone else?,Do you feel that you are trying to control someone?,"Do you see yourself as the victim, blaming someone else for how negative you feel?",Are you always blaming and accusing yourself for when something goes wrong?,Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,Was this emotion triggered by a specific event?,Was the event recent?,Did you find protocol 6 distressing?,...,"ppl_Do you see yourself as the victim, blaming someone else for how negative you feel?",ppl_Are you always blaming and accusing yourself for when something goes wrong?,ppl_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,ppl_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,ppl_Was this emotion triggered by a specific event?,ppl_Was the event recent?,ppl_Did you find protocol 6 distressing?,ppl_Did you find protocol 11 distressing?,ppl_Is it ok to ask additional questions?,ppl_Would you like to attempt protocols for joy?
0,"Спасибо, что вы здесь с нами.как вы считаете и...","Я понимаю, это не самый легкий вопрос.Бывает л...","спасибо за откровенность.Скажите пожалуйста, в...",Благодарю за открытость.Возможно вы оказались ...,Я понимаю вопрос не из лёгких.Как часто вы руг...,Спасибо за доверие.Вы часто настаиваете на том...,"Понимаю, что на этот вопрос нелегко ответить.Б...","Ценю Вашу открытость.Можете ли вы вспомнить, в...","Спасибо, что обратились именно ко мне. Ваше чу...","Спасибо,что вы поделились со мной своими ощуще...",...,0.001831,0.002969,0.001470,0.001749,0.000483,0.007997,0.004818,0.004552,0.000982,0.0
1,"Я рад, что вы мне доверяете.как вы считаете ис...","Спасибо за доверие.Бывает ли у Вас чувство, чт...","Я понимаю, это не самый легкий вопрос.Скажите ...",Спасибо за доверие.Возможно вы оказались в пол...,"Спасибо за доверие.Как часто вы ругаете себя, ...",Спасибо за искренность.Вы часто настаиваете на...,Благодарю вас.Благодарю вас за оказанное довер...,"Спасибо, что поделились своими чувствами.Может...",Я очень рада что ты решил поделиться со мной с...,"Спасибо, продолжим наш диалог. Возникли ли у В...",...,0.000854,0.002858,0.002253,0.000788,0.003955,0.000538,0.002525,0.002627,0.003417,0.0
2,По всей видимости это нелегко. Испытывали ли в...,"Не кажется ли вам, что вы недостаточно контрол...","Наш с Вами разговор будет более плодотворным, ...","Спасибо, что были открыты со мной. И если у Ва...","Я понимаю, что это трудно.Когда у вас что-то н...","Я думаю это непростой вопрос, но все же попыта...","Спасибо, что доверились мне. Я бы хотел узнат...","Задумайтесь, пожалуста, над истоком Вашего чув...","Я очень вас понимаю и сочувствую, и постараюсь...",Я очень вас понимаю и хочу помочь вам. Были ли...,...,0.003946,0.003594,0.002657,0.001213,0.005597,0.002231,0.001338,0.002394,0.001411,0.0
3,Благодарю за искренность. Испытывали ли вы к к...,"Спасибо, что верите мне. А возникает ли у вас ...","Я очень рад, что вы мне доверились.Нет ли у ва...",Проанализируйте пожалуйста и откровенно ответь...,Я задам Вам непростой вопрос.Когда у вас что-т...,"Благодарю за искренность. Случается ли у Вас, ...","Не бывает ли у вас ""комплекса героя"", желания ...","Спасибо ща ответ.Вы полагаете , что чувство тр...",Я понимаю Вас большего кто бы то не был. А это...,"Благодарен за Ваше доверие и откровенность, оч...",...,0.002390,0.001445,0.001815,0.001003,0.004986,0.001027,0.004173,0.001045,0.001613,0.0
4,Огромное спасибо за откровение. Вы ранее испыт...,"Спасибо за откровенность. Бывает ли такое, что...","Спасибо за то, что открылись мне.Бывало ли у в...",Не просто об этом говорить.У вас возникало чув...,Благодарю за доверие.Вам приходится обвинять с.

In [ ]:
df[df['emotion']=='fear']['ppl_Did you find protocol 6 distressing?'].max()

0.021732128863074837

In [ ]:
len(df.emotion.unique())
max_ppl = dict()
for emotion_type in df.emotion.unique():
  max_ppl.setdefault(emotion_type,[])
  for column in perplexity_columns:
    val = df[df['emotion']==emotion_type][f"ppl_{column}"].max()
    lst = max_ppl.get(emotion_type)
    lst.append(val)
    max_ppl[emotion_type] = lst

In [ ]:
max_ppl['fear']


[0.031132582112209074,
 0.10430648624925344,
 0.05306057636460039,
 0.026717013822536467,
 0.046898865644063724,
 0.023891980076649305,
 0.009476923317192128,
 0.03454735276744928,
 0.028499792674619176,
 0.021732128863074837,
 0.023437717233003957,
 0.02357366401305875,
 0.0]

In [ ]:
def normalise_ppl(row, column, largest_ppl):
  row[column] = row[column]/largest_ppl
  return row


In [ ]:
ds.reset_format()

In [ ]:
normalised_ds = ds
for index, column in enumerate(perplexity_columns):
  normalised_ds = normalised_ds.map(lambda x: normalise_ppl(x, f"ppl_{column}", largest_ppl= max_ppl[x['emotion']][index]))

  0%|          | 0/786 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

...

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

In [ ]:
# epru[train]["ppl_Did you find protocol 6 distressing?"]
normalised_ds["ppl_Have you strongly felt or expressed any of the following emotions towards someone?"]

0      0.068470
1      0.016836
2      0.053924
3      0.124455
4      0.067219
         ...   
781         NaN
782         NaN
783         NaN
784         NaN
785         NaN
Name: ppl_Have you strongly felt or expressed any of the following emotions towards someone?, Length: 786, dtype: float64

## Empathy

In [ ]:
model_id = "mmillet/distilrubert-tiny-cased-conversational-v1_single_finetuned_empathy_classifier"
classifier = pipeline('text-classification', model=model_id)

In [ ]:
preds = classifier(ds[8]["Have you strongly felt or expressed any of the following emotions towards someone?"], top_k=3)
preds = classifier("Это очень хорошо, что вы открыты, очень постараюсь вам помочь. Уверен, что у нас все получится. Задам вам дополнительные вопросы.", top_k=3)
print(preds)
prediction = max(preds, key=lambda x: x['score'])
print(prediction)

[{'label': 'LABEL_0', 'score': 0.8753650784492493}, {'label': 'LABEL_2', 'score': 0.11078225821256638}, {'label': 'LABEL_1', 'score': 0.01385271642357111}]
{'label': 'LABEL_0', 'score': 0.8753650784492493}


In [ ]:
ds[8]["Have you strongly felt or expressed any of the following emotions towards someone?"]

'Я хочу задать не простой вопрос. Вы испытывали к кому-то злость, ненависть, зависть?'

In [ ]:
normalised_ds.reset

In [ ]:
pipeline('text-classification', model=self.model_id)

preds = self.classifier(text, top_k=3)
prediction = max(preds, key=lambda x: x['score'])
prediction_label = self.int2label(int(prediction['label'][-1]))

NameError: ignored

In [ ]:
def calculate_empathy(row, column_name):
  if row[column_name]!= None: #len(row[column_name]) != 0:
    preds = classifier(row[column_name], top_k=3)
    prediction = max(preds, key=lambda x: x['score'])
    if prediction["label"] == 'LABEL_0':
      score = 1
    elif prediction["label"] == 'LABEL_1':
      score = 0.33
    else:
      score = 0.67
    return {f'emp_{column_name}': score}
  else:
    return {f'emp_{column_name}': 0}

In [ ]:
for column_name in perplexity_columns:
  normalised_ds = normalised_ds.map(lambda x: calculate_empathy(x, column_name))

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

...

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

  0%|          | 0/786 [00:00<?, ?ex/s]

In [ ]:
# change dataset into pandas 
normalised_ds.set_format(type="pandas")
df = normalised_ds[:]
# df["emp_Have you strongly felt or expressed any of the following emotions towards someone?"]
df.head()

,Have you strongly felt or expressed any of the following emotions towards someone?,Do you believe you should be the savior of someone else?,Do you feel that you are trying to control someone?,"Do you see yourself as the victim, blaming someone else for how negative you feel?",Are you always blaming and accusing yourself for when something goes wrong?,Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,Was this emotion triggered by a specific event?,Was the event recent?,Did you find protocol 6 distressing?,...,"emp_Do you see yourself as the victim, blaming someone else for how negative you feel?",emp_Are you always blaming and accusing yourself for when something goes wrong?,emp_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,emp_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,emp_Was this emotion triggered by a specific event?,emp_Was the event recent?,emp_Did you find protocol 6 distressing?,emp_Did you find protocol 11 distressing?,emp_Is it ok to ask additional questions?,emp_Would you like to attempt protocols for joy?
0,"Спасибо, что вы здесь с нами.как вы считаете и...","Я понимаю, это не самый легкий вопрос.Бывает л...","спасибо за откровенность.Скажите пожалуйста, в...",Благодарю за открытость.Возможно вы оказались ...,Я понимаю вопрос не из лёгких.Как часто вы руг...,Спасибо за доверие.Вы часто настаиваете на том...,"Понимаю, что на этот вопрос нелегко ответить.Б...","Ценю Вашу открытость.Можете ли вы вспомнить, в...","Спасибо, что обратились именно ко мне. Ваше чу...","Спасибо,что вы поделились со мной своими ощуще...",...,0.67,0.67,1.00,1.00,0.33,1.00,1.00,1.0,0.33,0.0
1,"Я рад, что вы мне доверяете.как вы считаете ис...","Спасибо за доверие.Бывает ли у Вас чувство, чт...","Я понимаю, это не самый легкий вопрос.Скажите ...",Спасибо за доверие.Возможно вы оказались в пол...,"Спасибо за доверие.Как часто вы ругаете себя, ...",Спасибо за искренность.Вы часто настаиваете на...,Благодарю вас.Благодарю вас за оказанное довер...,"Спасибо, что поделились своими чувствами.Может...",Я очень рада что ты решил поделиться со мной с...,"Спасибо, продолжим наш диалог. Возникли ли у В...",...,1.00,1.00,1.00,1.00,1.00,1.00,0.67,1.0,1.00,0.0
2,По всей видимости это нелегко. Испытывали ли в...,"Не кажется ли вам, что вы недостаточно контрол...","Наш с Вами разговор будет более плодотворным, ...","Спасибо, что были открыты со мной. И если у Ва...","Я понимаю, что это трудно.Когда у вас что-то н...","Я думаю это непростой вопрос, но все же попыта...","Спасибо, что доверились мне. Я бы хотел узнат...","Задумайтесь, пожалуста, над истоком Вашего чув...","Я очень вас понимаю и сочувствую, и постараюсь...",Я очень вас понимаю и хочу помочь вам. Были ли...,...,1.00,0.67,0.67,1.00,1.00,1.00,1.00,1.0,1.00,0.0
3,Благодарю за искренность. Испытывали ли вы к к...,"Спасибо, что верите мне. А возникает ли у вас ...","Я очень рад, что вы мне доверились.Нет ли у ва...",Проанализируйте пожалуйста и откровенно ответь...,Я задам Вам непростой вопрос.Когда у вас что-т...,"Благодарю за искренность. Случается ли у Вас, ...","Не бывает ли у вас ""комплекса героя"", желания ...","Спасибо ща ответ.Вы полагаете , что чувство тр...",Я понимаю Вас большего кто бы то не был. А это...,"Благодарен за Ваше доверие и откровенность, оч...",...,0.33,0.33,1.00,0.33,0.67,0.67,1.00,1.0,1.00,0.0
4,Огромное спасибо за откровение. Вы ранее испыт...,"Спасибо за откровенность. Бывает ли такое, что...","Спасибо за то, что открылись мне.Бывало ли у в...",Не просто об этом говорить.У вас возникало чув...,Благодарю за доверие.Вам приходится обвинять с...,Благодарю за доверие. Как часто Вы не слышите ...,"Спасибо, что были открыты со мной. Хотел поинт...","Спасибо, что уточнили Ваше ощущение.Чувств

In [ ]:
normalised_ds.reset_format()

## Generate embeddings of emotional utterances

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
pipe = pipeline("feature-extraction", model=model, tokenizer=tokenizer)
features = pipe("Чувствую себя очень плохо. Мне очень грустно.")

In [ ]:
def generate_embeddings(row):
  embedding = pipe(row['How are you feeling?'])
  return {'embeddings': embedding}

In [ ]:
normalised_ds_with_embeddings= normalised_ds.map(generate_embeddings)

  0%|          | 0/340 [00:00<?, ?ex/s]

KeyError: ignored

# Save dataset

In [ ]:
normalised_ds.save_to_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset/ppl_empathy_added")

# Checking dataset

In [ ]:
from datasets.load import load_from_disk
ds = load_from_disk("/content/drive/MyDrive/chatbot/SyntheticResponseGenerationDataset/ppl_empathy_added")

In [ ]:
ds

Dataset({
    features: ['Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'emotion', 'Would you like to attempt protocols for joy?', 'ppl_Have you strongly felt or expressed any of the following emotions towards someone?', 'ppl_Do you believe you should be the s